In [163]:
import keras
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.callbacks import *
from keras.preprocessing.image import *

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [165]:
# display model as svg
def model_as_svg(m):
    from IPython.display import SVG
    from keras.utils.vis_utils import model_to_dot
    return SVG(model_to_dot(m, show_shapes=True)\
               .create(prog='dot', format='svg'))

In [166]:
N_CLASSES = 10

In [167]:
# load data
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

# convert ys to one-hots
y_train = keras.utils.to_categorical(y_train, num_classes=N_CLASSES)
y_test = keras.utils.to_categorical(y_test, num_classes=N_CLASSES)

# normalize data

mean_X = np.mean(X_train, axis=0)
X_train -= mean_X
X_test -= mean_X

std_X = np.std(X_test)
X_train /= std_X
X_test /= std_X

In [172]:
def build_plain_cnn_1(input_shape=(32, 32, 3), n_clases=10):

    i = Input(shape=input_shape, name='input')

    x = BatchNormalization()(i)

    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(n_clases, activation='softmax')(x)

    return Model(inputs=[i], outputs=[x])

In [173]:
def build_resnet18(input_size=(32, 32, 3), n_classes=10):

    i = Input(shape=input_size, name='input')

    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='conv_0')(i)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    y = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), padding='same', name='conv_64_1a')(y)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same', name='conv_64_1b')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    y = Add()([x, y])
    y = Activation('relu')(y)

    x = Conv2D(64, (3, 3), padding='same', name='conv_64_2a')(y)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, (3, 3), padding='same', name='conv_64_2b')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    y = Add()([x, y])
    y = Activation('relu')(y)

    x = Conv2D(
        128, (3, 3), strides=(2, 2), padding='same', name='conv_128_1a')(y)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3, 3), padding='same', name='conv_128_1b')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    y = Conv2D(128, (3, 3), strides=(2, 2), padding='same')(y)
    y = BatchNormalization()(y)
    y = Add()([x, y])
    y = Activation('relu')(y)

    x = Conv2D(128, (3, 3), padding='same', name='conv_128_2a')(y)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128, (3, 3), padding='same', name='conv_128_2b')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    y = Add()([x, y])
    y = Activation('relu')(y)

    x = Conv2D(
        256, (3, 3), strides=(2, 2), padding='same', name='conv_256_1a')(y)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same', name='conv_256_1b')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    y = Conv2D(256, (3, 3), strides=(2, 2), padding='same')(y)
    y = BatchNormalization()(y)
    y = Add()([x, y])
    y = Activation('relu')(y)

    x = Conv2D(256, (3, 3), padding='same', name='conv_256_2a')(y)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256, (3, 3), padding='same', name='conv_256_2b')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    y = Add()([x, y])
    y = Activation('relu')(y)

    x = Conv2D(
        512, (3, 3), strides=(2, 2), padding='same', name='conv_512_1a')(y)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv_512_1b')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    y = Conv2D(512, (3, 3), strides=(2, 2), padding='same')(y)
    y = BatchNormalization()(y)
    y = Add()([x, y])
    y = Activation('relu')(y)

    x = Conv2D(512, (3, 3), padding='same', name='conv_512_2a')(y)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512, (3, 3), padding='same', name='conv_512_2b')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    y = Add()([x, y])
    y = Activation('relu')(y)

    # add average poling as needed to bring filter size to 1x1
    # (on ImageNet 224x224 input filter size here is 7x7, on 32x32 for CIFAR it's 1x1 already)
    y_filter_size = y.shape.as_list()[1:3]
    if max(y_filter_size) > 1:
        x = AveragePooling2D(tuple(y_filter_size))(y)

    x = Flatten()(y)
    x = Dense(n_classes, activation='softmax')(x)

    return Model(inputs=[i], outputs=[x])

In [194]:
model = build_resnet18()
# model = build_plain_cnn_1()

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(lr=1e-3),
    metrics=['accuracy'])

AUGMENTATION = True
BATCH_SIZE = 256
EPOCHS = 100
SAMPLES_PER_EPOCH = len(X_train) // BATCH_SIZE * BATCH_SIZE
print('SAMPLES_PER_EPOCH:', SAMPLES_PER_EPOCH, 'of', len(X_train))

callbacks = [
    ReduceLROnPlateau(
        factor=0.333,
        cooldown=0,
        patience=2,
        min_lr=1e-9,
        verbose=True,
        monitor='val_acc'),
    EarlyStopping(
        min_delta=0.001, patience=10, monitor='val_acc', verbose=True)
]

if AUGMENTATION:

    g = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=0.,
        width_shift_range=0.25,
        height_shift_range=0.25,
        horizontal_flip=True,
        vertical_flip=False)

    g.fit(X_train)

    model.fit_generator(
        g.flow(X_train, y_train, batch_size=BATCH_SIZE, shuffle=True),
        steps_per_epoch=SAMPLES_PER_EPOCH // BATCH_SIZE,
        validation_data=(X_test, y_test),
        epochs=EPOCHS,
        verbose=1,
        max_queue_size=100,
        callbacks=callbacks)

else:

    model.fit(
        X_train,
        y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_test, y_test),
        shuffle=True,
        callbacks=callbacks)

SAMPLES_PER_EPOCH: 49920 of 50000
Epoch 1/100
195/195 [==============================] - 13s - loss: 1.9017 - acc: 0.3465 - val_loss: 2.6356 - val_acc: 0.2595
Epoch 2/100
195/195 [==============================] - 10s - loss: 1.4769 - acc: 0.4786 - val_loss: 1.7127 - val_acc: 0.4994
Epoch 3/100
195/195 [==============================] - 10s - loss: 1.2804 - acc: 0.5462 - val_loss: 1.3760 - val_acc: 0.5210
Epoch 4/100
195/195 [==============================] - 10s - loss: 1.1823 - acc: 0.5852 - val_loss: 2.3478 - val_acc: 0.4452
Epoch 5/100
195/195 [==============================] - 10s - loss: 1.0856 - acc: 0.6204 - val_loss: 1.2669 - val_acc: 0.5871
Epoch 6/100
195/195 [==============================] - 10s - loss: 0.9964 - acc: 0.6505 - val_loss: 1.1076 - val_acc: 0.6321
Epoch 7/100
195/195 [==============================] - 10s - loss: 0.9235 - acc: 0.6753 - val_loss: 1.4291 - val_acc: 0.5585
Epoch 8/100
195/195 [==============================] - 10s - loss: 0.9127 - acc: 0.6860 - v

195/195 [==============================] - 10s - loss: 0.3006 - acc: 0.8939 - val_loss: 0.4707 - val_acc: 0.8506
Epoch 58/100
195/195 [==============================] - 10s - loss: 0.2978 - acc: 0.8947 - val_loss: 0.4710 - val_acc: 0.8507
Epoch 59/100
194/195 [============================>.] - ETA: 0s - loss: 0.3039 - acc: 0.8919
Epoch 00058: reducing learning rate to 1.512007213193556e-07.
195/195 [==============================] - 10s - loss: 0.3036 - acc: 0.8919 - val_loss: 0.4706 - val_acc: 0.8507
Epoch 60/100
195/195 [==============================] - 10s - loss: 0.3015 - acc: 0.8933 - val_loss: 0.4710 - val_acc: 0.8512
Epoch 61/100
194/195 [============================>.] - ETA: 0s - loss: 0.3033 - acc: 0.8933
Epoch 00060: reducing learning rate to 5.03498407766756e-08.
195/195 [==============================] - 10s - loss: 0.3033 - acc: 0.8934 - val_loss: 0.4708 - val_acc: 0.8505
Epoch 00060: early stopping
